In [2]:
# various imports and setup

import os
import json
import requests
from zipfile import ZipFile

from tqdm import tqdm

In [4]:
data_dir = 'data/openfda/'
zip_dir = data_dir + 'zip/'
json_dir = data_dir + 'json/'

json_print = lambda j: print(json.dumps(j, indent=4, sort_keys=True))
RUN_DIAGNOSTICS = False

#%% download raw data (11 zip files)

dl_json_url = 'https://api.fda.gov/download.json'
dl_json = json.loads(requests.get(dl_json_url, verify=False).text)
json_print(dl_json['meta'])
print(f"Overall categories: {[k for k in dl_json['results'].keys()]}")

drugs_json = dl_json['results']['drug']
print(f"Drug data categories: {[k for k in drugs_json.keys()]}")

labels_json = dl_json['results']['drug']['label']
print(f"\nFound {labels_json['total_records']} labels, last updated {labels_json['export_date']}\n")

urls = [x['file'] for x in labels_json['partitions']]

if not os.path.exists(zip_dir):
    os.mkdir(zip_dir)

for url in urls:
    filename = url.split('/')[-1]
    if os.path.exists(zip_dir+filename):
        print(f"Already downloaded file {url}")
    else:
        r = requests.get(url, verify=False)      
        with open(zip_dir+filename, 'wb') as f:
            f.write(r.content)
            print(f'Downloaded file {url}')
                  

#%% unzip those files (result is several jsons)

if not os.path.exists(json_dir):
      os.mkdir(json_dir)

for file in os.listdir(zip_dir):
    with ZipFile(zip_dir + file, 'r') as zf:  
        for zobj in zf.infolist():
            if os.path.exists(json_dir + zobj.filename):
                print(f'Already extracted file {zobj.filename}')
            else:
                zf.extract(zobj, json_dir)
                print(f'Extracted file {zobj.filename}')


#%% read all extracted jsons into master list

results = []
    
for file in tqdm(os.listdir(json_dir)):
    with open(json_dir + file, encoding='utf-8') as f:
        j = json.load(f)
        results += j['results']


#%% loop through master list, sort results into rx/otc/other

# note: many "uncategorized" drugs are simply missing their metadata,
#  which is usually stored in the "openfda" field (a dict)
def check_type(res):
    if 'product_type' not in res['openfda'].keys():
        return 'uncategorized_drug'
    pt = res['openfda']['product_type']
    if type(pt)==float:
        return 'uncategorized_drug'
    if type(pt)==list:
        assert(len(pt)==1)
        return(pt[0].lower().replace(' ','_'))
    else:
        print('Problem determining type: ', pt)
        
results_by_type = {}
for res in results:
    product_type = check_type(res)
    if product_type not in results_by_type.keys():
        results_by_type[product_type] = [res]
    else:
        results_by_type[product_type].append(res)

        
#%% we only care about the prescription drugs, save to disk

drugs = results_by_type['human_prescription_drug']

errors = []
records = {}
# restructure to match ema format and save to disk
for drug in tqdm(drugs):
    try:
        info = {}
        info['metadata'] = drug['openfda']
        
        label_text = {}
        for key,val in drug.items():
            # for my purposes I didn't need tables (mostly html formatting)
            if (type(val)==list) and ('table' not in key):
                label_text[key] = list(set(val)) # de-duplicate contents
        info['Label Text'] = label_text
        
        records[drug['id']] = info

    except:
        errors += [drug]

print(f'Encountered problems reading {len(errors)} records')
with open('data/output/human-rx-openfda-drug.json', 'w') as f:
    json.dump(records, f, indent=4)

# for manual inspection, save the first few separately (full file too big)
if RUN_DIAGNOSTICS:
    with open('data/output/human-rx-drug-openfda-excerpt.json', 'w') as f:
        json.dump({k:records[k] for k in list(records.keys())[:100]}, f, indent=4)



/Users/colecrawford/.pyenv/versions/3.10.0/lib/python3.10/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.fda.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{
    "disclaimer": "Do not rely on openFDA to make decisions regarding medical care. While we make every effort to ensure that data is accurate, you should assume all results are unvalidated. We may limit or otherwise restrict your access to the API in line with our Terms of Service.",
    "last_updated": "2023-03-25",
    "license": "https://open.fda.gov/license/",
    "terms": "https://open.fda.gov/terms/"
}
Overall categories: ['drug', 'device', 'food', 'other', 'tobacco', 'animalandveterinary']
Drug data categories: ['event', 'label', 'enforcement', 'ndc', 'drugsfda']

Found 209614 labels, last updated 2023-03-25



/Users/colecrawford/.pyenv/versions/3.10.0/lib/python3.10/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'download.open.fda.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Downloaded file https://download.open.fda.gov/drug/label/drug-label-0001-of-0011.json.zip


/Users/colecrawford/.pyenv/versions/3.10.0/lib/python3.10/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'download.open.fda.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


...

Downloaded file https://download.open.fda.gov/drug/label/drug-label-0010-of-0011.json.zip


/Users/colecrawford/.pyenv/versions/3.10.0/lib/python3.10/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'download.open.fda.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Downloaded file https://download.open.fda.gov/drug/label/drug-label-0011-of-0011.json.zip
Extracted file drug-label-0004-of-0011.json
Extracted file drug-label-0010-of-0011.json
Extracted file drug-label-0005-of-0011.json
Extracted file drug-label-0011-of-0011.json
Extracted file drug-label-0001-of-0011.json
Extracted file drug-label-0006-of-0011.json
Extracted file drug-label-0002-of-0011.json
Extracted file drug-label-0009-of-0011.json
Extracted file drug-label-0007-of-0011.json
Extracted file drug-label-0003-of-0011.json
Extracted file drug-label-0008-of-0011.json


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32103/32103 [00:13<00:00, 2321.62it/s]


Encountered problems reading 0 records


In [3]:
with open("data/output/human-rx-openfda-drug.json") as f:
    data_fda = json.load(f)

In [6]:
data_fda['4fdf3b7e-f6ff-4d78-a928-c9d47c5bc9d9']['metadata']

{'application_number': ['ANDA210175'],
 'brand_name': ['OXYMORPHONE HYDROCHLORIDE'],
 'generic_name': ['OXYMORPHONE HYDROCHLORIDE'],
 'manufacturer_name': ['XLCare Pharmaceuticals, Inc.'],
 'product_ndc': ['72865-130', '72865-131'],
 'product_type': ['HUMAN PRESCRIPTION DRUG'],
 'route': ['ORAL'],
 'substance_name': ['OXYMORPHONE HYDROCHLORIDE'],
 'rxcui': ['977939', '977942'],
 'spl_id': ['4fdf3b7e-f6ff-4d78-a928-c9d47c5bc9d9'],
 'spl_set_id': ['1a1d5061-b62a-49fa-a5b2-4970f51416ae'],
 'package_ndc': ['72865-130-01', '72865-131-01'],
 'is_original_packager': [True],
 'upc': ['0372865131015', '0372865130018'],
 'unii': ['5Y2EI94NBC']}

In [11]:
data_fda['4fdf3b7e-f6ff-4d78-a928-c9d47c5bc9d9'].keys()

dict_keys(['metadata', 'Label Text'])